In [387]:
# !pip install pdfplumber langchain openai pandas scikit-learn

In [388]:
import os
import re
import time
from typing import List, Dict, Tuple, Optional, Any
from openai import OpenAI
# from pypdf import PdfReader # No longer needed
import pdfplumber # Use pdfplumber instead
from dotenv import load_dotenv
import json
# Add LangChain import
from langchain_text_splitters import RecursiveCharacterTextSplitter
import pandas as pd
import datetime


In [1]:
from markitdown import MarkItDown

md = MarkItDown(enable_plugins=False)
result = md.convert("10K-filings/10K-filings/AAPL_10k.pdf")
print(result.text_content)

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, def

UNITED STATES
SECURITIES AND EXCHANGE COMMISSION
Washington, D.C. 20549

FORM 10-K

(Mark One)

☒ ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934
For the fiscal year ended September 28, 2024
or
☐ TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934

For the transition period from              to             .

Commission File Number: 001-36743

Apple Inc.

(Exact name of Registrant as specified in its charter)

California
(State or other jurisdiction
of incorporation or organization)

One Apple Park Way
Cupertino, California
(Address of principal executive offices)

94-2404110
(I.R.S. Employer Identification No.)

95014
(Zip Code)

(408) 996-1010
(Registrant’s telephone number, including area code)

Securities registered pursuant to Section 12(b) of the Act:

Title of each class
Common Stock, $0.00001 par value per share

0.000% Notes due 2025
0.875% Notes due 2025
1.625% Notes due 2026
2.000% Notes due 2027
1.375

In [389]:
def extract_text_from_pdf(pdf_path: str) -> str:
    """Extracts text content from a PDF file using pdfplumber with improved table handling."""
    print(f"Extracting text from: {pdf_path} using pdfplumber")
    text = ""
    tables_found = 0
    
    try:
        with pdfplumber.open(pdf_path) as pdf:
            for i, page in enumerate(pdf.pages):
                # Extract regular text
                page_text = page.extract_text(x_tolerance=2, y_tolerance=2) 
                
                # Extract tables and convert to text representation
                tables = page.extract_tables()
                table_text = ""
                
                if tables:
                    tables_found += len(tables)
                    for table_idx, table in enumerate(tables):
                        # Add table marker and metadata
                        table_text += f"\n--- TABLE {i+1}-{table_idx+1} ---\n"
                        
                        # Convert table to plain text with better formatting
                        for row in table:
                            # Replace None with empty string and join with tabs
                            formatted_row = "\t".join([str(cell) if cell is not None else "" for cell in row])
                            table_text += formatted_row + "\n"
                        
                        table_text += f"--- END TABLE {i+1}-{table_idx+1} ---\n\n"
                
                # Combine page text and table text
                if page_text:
                    text += page_text + "\n"
                if table_text:
                    text += table_text
                
                if (i + 1) % 10 == 0:
                    print(f"  Processed page {i+1}/{len(pdf.pages)}")

        print(f"Successfully extracted ~{len(text)} characters including {tables_found} tables.")
        text = re.sub(r'\n{3,}', '\n\n', text).strip()  # Clean up excessive newlines but preserve paragraph breaks
        return text
    except Exception as e:
        print(f"Error reading PDF {pdf_path} with pdfplumber: {e}")
        return ""

In [390]:
text = extract_text_from_pdf("10K-filings/10K-filings/AAPL_10k.pdf")


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, def

Extracting text from: 10K-filings/10K-filings/AAPL_10k.pdf using pdfplumber


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, def

  Processed page 10/60
  Processed page 20/60
  Processed page 30/60
  Processed page 40/60
  Processed page 50/60


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, def

  Processed page 60/60
Successfully extracted ~225808 characters including 50 tables.


In [391]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

def chunk_text_with_tables(text: str, chunk_size: int = 2500) -> List[str]:
    """Splits text into chunks while preserving table structures."""
    print(f"Chunking text using table-aware RecursiveCharacterTextSplitter (chunk size: {chunk_size})...")

    # Define splitting logic that preserves tables
    # The recursive splitter tries these separators in order
    # We want tables to be kept intact, so we ensure table markers aren't used as split points
    separators = [
        "\n\n\n",              # Large paragraph breaks (highest priority)
        "\n\n",                # Normal paragraph breaks
        "\n",                  # Line breaks (inside paragraphs)
        ". ",                  # Sentence breaks
        ", ",                  # Clause breaks
        " ",                   # Word breaks (lowest priority)
        ""                     # Character breaks (if all else fails)
    ]
    
    # First identify table sections to protect them
    table_markers = re.findall(r'--- TABLE \d+-\d+ ---.*?--- END TABLE \d+-\d+ ---', text, re.DOTALL)
    
    # If there are very large tables, we might need to split them individually
    large_tables = []
    for table in table_markers:
        if len(table) > chunk_size * 0.8:  # If table is more than 80% of chunk size
            large_tables.append(table)
    
    # Replace large tables with placeholders
    protected_text = text
    placeholders = {}
    for i, table in enumerate(large_tables):
        placeholder = f"[TABLE_PLACEHOLDER_{i}]"
        placeholders[placeholder] = table
        protected_text = protected_text.replace(table, placeholder)
    
    # Configure the splitter
    text_splitter = RecursiveCharacterTextSplitter(
        separators=separators,
        chunk_size=chunk_size,
        chunk_overlap=int(chunk_size * 0.15),  # 10% overlap
        length_function=len,
    )
    
    # Split the text (with large tables as placeholders)
    chunks = text_splitter.split_text(protected_text)
    
    # Process large tables separately with higher chunk size to keep more structure
    table_chunks = []
    for table in large_tables:
        table_splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size * 2,  # Use larger chunks for tables
            chunk_overlap=int(chunk_size * 0.2),  # More overlap for tables
            length_function=len,
        )
        table_chunks.extend(table_splitter.split_text(table))
    
    # Restore placeholders and add table chunks
    final_chunks = []
    for chunk in chunks:
        # Check if chunk contains any table placeholders
        has_placeholder = False
        for placeholder, table in placeholders.items():
            if placeholder in chunk:
                has_placeholder = True
                # Replace with original table content if small enough
                if len(table) <= chunk_size:
                    chunk = chunk.replace(placeholder, table)
                else:
                    # Remove placeholder to prevent confusion
                    chunk = chunk.replace(placeholder, 
                                         "[Table was too large and is processed separately]")
        
        if chunk.strip():  # Avoid empty chunks
            final_chunks.append(chunk)
    
    # Add the separate table chunks    final_chunks.extend(table_chunks)
    
    print(f"Text split into {len(final_chunks)} chunks, including special handling for tables.")
    return final_chunks

In [392]:
chunks = chunk_text_with_tables(text)
print(chunks)

Chunking text using table-aware RecursiveCharacterTextSplitter (chunk size: 2500)...
Text split into 123 chunks, including special handling for tables.
['UNITED STATES\nSECURITIES AND EXCHANGE COMMISSION\nWashington, D.C. 20549\nFORM 10-K\n(Mark One)\n☒ ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934\nFor the fiscal year ended September 28, 2024\nor\n☐ TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934\nFor the transition period from to .\nCommission File Number: 001-36743\nApple Inc.\n(Exact name of Registrant as specified in its charter)\nCalifornia 94-2404110\n(State or other jurisdiction (I.R.S. Employer Identification No.)\nof incorporation or organization)\nOne Apple Park Way\nCupertino, California 95014\n(Address of principal executive offices) (Zip Code)\n(408) 996-1010\n(Registrant’s telephone number, including area code)\nSecurities registered pursuant to Section 12(b) of the Act:\nTrading\nTitle of ea

In [393]:
chunks[0]

'UNITED STATES\nSECURITIES AND EXCHANGE COMMISSION\nWashington, D.C. 20549\nFORM 10-K\n(Mark One)\n☒ ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934\nFor the fiscal year ended September 28, 2024\nor\n☐ TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934\nFor the transition period from to .\nCommission File Number: 001-36743\nApple Inc.\n(Exact name of Registrant as specified in its charter)\nCalifornia 94-2404110\n(State or other jurisdiction (I.R.S. Employer Identification No.)\nof incorporation or organization)\nOne Apple Park Way\nCupertino, California 95014\n(Address of principal executive offices) (Zip Code)\n(408) 996-1010\n(Registrant’s telephone number, including area code)\nSecurities registered pursuant to Section 12(b) of the Act:\nTrading\nTitle of each class symbol(s) Name of each exchange on which registered\nCommon Stock, $0.00001 par value per share AAPL The Nasdaq Stock Market LLC\n0.000% Notes d

In [394]:
chunks[1]

'Yes ☒ No ☐\nIndicate by check mark whether the Registrant has submitted electronically every Interactive Data File required to be submitted pursuant to Rule 405\nof Regulation S-T (§232.405 of this chapter) during the preceding 12 months (or for such shorter period that the Registrant was required to submit\nsuch files).\nYes ☒ No ☐\nIndicate by check mark whether the Registrant is a large accelerated filer, an accelerated filer, a non-accelerated filer, a smaller reporting company, or\nan emerging growth company. See the definitions of “large accelerated filer,” “accelerated filer,” “smaller reporting company,” and “emerging growth\ncompany” in Rule 12b-2 of the Exchange Act.\nLarge accelerated filer ☒ Accelerated filer ☐\n☐ ☐\nNon-accelerated filer Smaller reporting company\n☐\nEmerging growth company\nIf an emerging growth company, indicate by check mark if the Registrant has elected not to use the extended transition period for complying with any\nnew or revised financial accounti

In [395]:
import pandas as pd
from IPython.display import display

def visualize_chunks(chunks):
    print("--- Chunk Details ---")
    if chunks:
        # Convert string chunks to dictionary format with chunk numbers
        chunks_data = [{"chunk_number": i+1, "text": chunk} for i, chunk in enumerate(chunks)]
        
        # Create a DataFrame for better visualization
        chunks_df = pd.DataFrame(chunks_data)
        chunks_df['word_count'] = chunks_df['text'].apply(lambda x: len(x.split()))
        
        # Display the DataFrame with selected columns
        display(chunks_df[['chunk_number', 'word_count', 'text']])
    else:
        print("No chunks were created (text might be shorter than chunk size).")
    print("-" * 25)

# Example usage:
# chunks = chunk_text_with_tables(your_text, chunk_size=525)


In [396]:
visualize_chunks(chunks)

--- Chunk Details ---


,chunk_number,word_count,text
0,1,406,UNITED STATES\nSECURITIES AND EXCHANGE COMMISS...
1,2,383,Yes ☒ No ☐\nIndicate by check mark whether the...
2,3,352,the Registrant’s most recently completed secon...
3,4,216,--- TABLE 3-1 ---\nItem 1.\tBusiness\t1\nItem ...
4,5,366,This Annual Report on Form 10-K (“Form 10-K”) ...
...,...,...,...
118,119,410,Incorporated by Reference\nFiling Date/\nExhib...
119,120,411,forms of global notes representing the 1.800% ...
120,121,393,"2032, 3.950% Notes due 2052 and 4.100% Notes d..."
121,122,389,10.14* Form of Restricted Stock Unit Award Agr...


-------------------------


In [397]:
extraction_system_prompt = """
You are an AI expert specialized in knowledge graph extraction from SEC 10-K filings. Your task is to identify and extract high-quality Subject-Predicate-Object (SPO) triples that are relevant for financial analysis and company understanding.

MANDATORY RULES FOR TRIPLE EXTRACTION:

1. RELEVANCE:
   - Only extract triples that are directly relevant to understanding the company's:
     * Business operations and strategy
     * Financial performance and position
     * Risk factors and challenges
     * Corporate governance
     * Key relationships (customers, suppliers, partners)
     * Regulatory compliance
     * Market position
     * Competitive advantages
     * Technology and innovation
     * Market trends and industry position

2. QUALITY CRITERIA:
   - Subjects must be specific named entities (e.g., 'apple inc.', not 'the company')
   - Predicates must be clear, active relationships (1-2 words maximum)
   - Objects must be specific, quantifiable, or measurable when possible
   - All values must be lowercase for consistency
   - Include specific dates, amounts, and percentages when available
   - Use standardized relationship terms for similar concepts

3. AVOID:
   - Generic or obvious statements that don't add analytical value
   - Redundant or duplicate information
   - Overly complex or compound relationships
   - Ambiguous or vague entities
   - Legal boilerplate text that doesn't convey meaningful information
   - Forward-looking statements without specific metrics
   - General industry information not specific to the company

4. PRIORITIZE:
   - Material financial metrics and KPIs
   - Specific risk factors with quantifiable impact
   - Strategic partnerships and major customer relationships
   - Market share and competitive position data
   - Regulatory compliance status and requirements
   - Corporate governance structure and changes
   - Technology investments and intellectual property
   - Supply chain relationships and dependencies
   - Geographic market presence and expansion


5. ENTITY TYPE GUIDELINES:
   - Identify the most specific and relevant type for each entity
   - Types should reflect the entity's role in the financial context
   - Create meaningful types based on the entity's nature and usage
   - Examples of common types (but not limited to):
     * Business entities (companies, subsidiaries, partners)
     * Financial instruments (stocks, bonds, derivatives)
     * Regulatory concepts (compliance statuses, requirements)
     * Business metrics (KPIs, financial measures)
     * Market elements (sectors, industries, regions)
     * Time-based elements (periods, dates, fiscal years)

6 RELATIONSHIP DISCOVERY:
   - Use natural, meaningful relationships that accurately describe the connection
   - Relationships should be:
     * Clear and concise (1-2 words)
     * Action-oriented
     * Specific to the financial domain
     * Reflective of the true nature of the connection
   - Avoid generic relationships like "is" or "has" when more specific ones are applicable

7 CONTEXTUAL UNDERSTANDING:
   - Consider the broader context of the 10-K section
   - Maintain consistency within related triples
   - Capture relationships that provide meaningful insights
   - Preserve important quantitative and qualitative information

8 FORMAT:
   {
       "triples": [
           {
               "subject": "specific entity name (lowercase)",
               "predicate": "active verb/phrase",
               "object": "specific measurable target"
           }
       ]
   }

9. QUALITY CRITERIA:
   - Each triple should provide unique, valuable insights
   - Entities should be specific and well-defined
   - Relationships should be meaningful and accurate
   - Context should be preserved where relevant
   - Quantitative information should be structured appropriately
TARGET: Extract ONLY 7-10 highest-quality triples per section that provide actionable insights for financial analysis and business understanding.

QUALITY CHECK: Before outputting each triple, verify:
1. Is it specific and measurable?
2. Does it add unique analytical value?
3. Is it relevant for financial/business analysis?
4. Can it be used for meaningful graph queries?
5. Is it based on factual, verifiable information?
"""
extraction_user_prompt_template = """
Please extract Subject-Predicate-Object (S-P-O) triples from the text below.

MANDATORY EXTRACTION RULES:
Extract meaningful knowledge graph triples from the following 10-K filing text. 

EXTRACTION GUIDELINES:

1. ENTITY UNDERSTANDING:
   - Identify key entities and their natural types
   - Consider the entity's role in the financial context
   - Preserve important attributes and metadata
   - Group related entities appropriately

2. RELATIONSHIP ANALYSIS:
   - Identify meaningful connections between entities
   - Use specific, action-oriented relationships
   - Consider temporal and causal relationships
   - Capture quantitative relationships where relevant

4. CONTEXT PRESERVATION:
   - Note the section and context of each triple
   - Maintain links between related information
   - Preserve temporal and conditional aspects
   - Include relevant qualifiers and conditions

5. RELEVANCE AND QUALITY:
   - Extract ONLY triples that provide meaningful insights for 10-K financial analysis
   - Each triple must contain specific, verifiable information
   - Focus on material information that impacts business value
   - Prioritize quantifiable metrics and relationships

6. FORMATTING REQUIREMENTS:
   - Predicates: Use 1-2 word active verbs/phrases only
   - Case: ALL values must be lowercase
   - Numbers: Preserve exact figures, dates, and percentages
   - Names: Use full entity names (e.g., 'apple inc.' not 'apple')
   - Consistency: Use standardized predicate terms across triples

7. RELATIONSHIP TYPES TO EXTRACT:
   - Financial metrics and performance indicators
   - Risk factors with specific impacts
   - Strategic partnerships and key relationships
   - Market position and competitive advantages
   - Regulatory compliance status
   - Corporate governance structures
   - Operational metrics and KPIs

8. QUALITY VALIDATION:
   - Each triple must be factual and verifiable
   - No generic or boilerplate statements
   - No forward-looking statements without metrics
   - No duplicate or redundant information
   - No vague or ambiguous relationships

Text to Process:
```text
{text_chunk}
```
Company Name: {company_name}

OUTPUT FORMAT:
Return ONLY a JSON object with 7-10 highest quality triples:
{{
    "triples": [
        {{
            "subject": "specific entity (lowercase)",
            "predicate": "active verb/phrase (1-2 words)",
            "object": "specific measurable target (lowercase)"
        }}
    ]
}}
"""


In [398]:
import os
from google import genai
from pydantic import BaseModel, Field
import json
from google.genai import types



In [399]:
import instructor

In [400]:
API_KEY = ""

In [401]:
# Initialize lists to store results and failures
all_extracted_triples = []
failed_chunks = []


In [402]:

# import google.generativeai as genai

In [403]:
# generation_config = genai.types.GenerationConfig(
# temperature=0.1, # Set desired temperature (0.0-1.0). 0.1 is low, for more deterministic output.
# response_mime_type="application/json", # Crucial for Gemini JSON mode
# top_p=0.95,
# )

In [404]:
generate_content_config = types.GenerateContentConfig(
        temperature=0.15,
        thinking_config = types.ThinkingConfig(
            thinking_budget=0,
        ),
        response_mime_type="application/json",
    )

In [405]:
client = genai.Client(api_key=API_KEY)
instructor_client = instructor.from_genai(client, mode=instructor.Mode.GENAI_TOOLS,config=generate_content_config)

In [406]:
# genai.configure(api_key=API_KEY)
        
# # Initialize Google's Gemini client
# instructor_client = instructor.from_gemini(
# client=genai.GenerativeModel(
#     model_name="models/gemini-2.0-flash",
#     generation_config=generation_config
# ),
# mode=instructor.Mode.GEMINI_JSON)

In [407]:
 #--- Knowledge Graph Schema ---
class Triple(BaseModel):
  """Represents a single knowledge graph triple (Subject-Predicate-Object)."""
  subject: str = Field(description="The entity or concept being described.")
  predicate: str = Field(description="The relationship or property connecting the subject and object.")
  object: str = Field(description="The entity, concept, or value related to the subject via the predicate.")

class TripleList(BaseModel):
    """A list of knowledge graph triples."""
    triples: List[Triple]

In [408]:
chunk_index = 0 
if chunk_index < len(chunks):
    chunk_info = chunks[chunk_index]
    chunk_text = chunk_info
    chunk_num = chunk_index + 1 
    
    print(f"\n--- Processing Chunk {chunk_num}/{len(chunks)} --- ")
    
    # 1. Format the User Prompt
    print("1. Formatting User Prompt...")
    user_prompt = extraction_user_prompt_template.format(text_chunk=chunk_text,company_name="APPLE INC")
    # print(f"   Formatted Prompt (Snippet): {user_prompt[:200]}...{user_prompt[-100:]}") # Optional: View prompt
    
    try:
        # 2. Make the API Call
        print("2. Sending request to LLM...")
        
        # Send request using instructor client
        response = instructor_client.chat.completions.create(
            model="gemini-2.0-flash",
            messages=[
                {"role": "system", "content": extraction_system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            response_model=TripleList,
            max_retries=3
        )
        
        print("   LLM response received.")
        
        # 3. Process Response
        print("3. Processing response...")
        
        # The response is already validated by instructor/pydantic
        valid_triples_in_chunk = []
        
        for triple in response.triples:
            # Add source chunk info
            triple_dict = triple.model_dump()
            triple_dict['chunk'] = chunk_num
            valid_triples_in_chunk.append(triple_dict)
        
        # --- Display Valid Triples from this Chunk --- 
        if valid_triples_in_chunk:
            print(f"--- Valid Triples Extracted (Chunk {chunk_num}) ---")
            display(pd.DataFrame(valid_triples_in_chunk))
            print("-" * 20)
            # Add to the main list
            all_extracted_triples.extend(valid_triples_in_chunk)
        else:
            print(f"--- No valid triples extracted from this chunk. ---")
            print("-" * 20)

    except Exception as e:
        error_message = str(e)
        print(f"   ERROR during API call: {error_message}")
        failed_chunks.append({
            'chunk_number': chunk_num,
            'error': f'API Error: {error_message}',
            'response': ''
        })

    # --- Update Running Total (Visual Feedback) ---
    print(f"--- Running Total Triples Extracted: {len(all_extracted_triples)} --- ")
    print(f"--- Failed Chunks So Far: {len(failed_chunks)} --- ")
        
else:
    print(f"Chunk index {chunk_index} is out of bounds (Total chunks: {len(chunks)}). Skipping.")

print("\nFinished processing this chunk.")




--- Processing Chunk 1/123 --- 
1. Formatting User Prompt...
2. Sending request to LLM...
   LLM response received.
3. Processing response...
--- Valid Triples Extracted (Chunk 1) ---


,subject,predicate,object,chunk
0,apple inc.,fiscal year ended,"september 28, 2024",1
1,apple inc.,commission file number,001-36743,1
2,apple inc.,state of incorporation,california,1
3,apple inc.,irs employer id,94-2404110,1
4,apple inc.,address,"one apple park way, cupertino, california 95014",1
5,apple inc.,telephone number,(408) 996-1010,1
6,"common stock, $0.00001 par value per share",trading symbol,aapl,1
7,"common stock, $0.00001 par value per share",exchange,the nasdaq stock market llc,1
8,0.000% notes due 2025,exchange,the nasdaq stock market llc,1


--------------------
--- Running Total Triples Extracted: 9 --- 
--- Failed Chunks So Far: 0 --- 

Finished processing this chunk.


In [409]:
all_triples = []
all_failures = []
all_triples.extend(valid_triples_in_chunk)
print(all_triples)

[{'subject': 'apple inc.', 'predicate': 'fiscal year ended', 'object': 'september 28, 2024', 'chunk': 1}, {'subject': 'apple inc.', 'predicate': 'commission file number', 'object': '001-36743', 'chunk': 1}, {'subject': 'apple inc.', 'predicate': 'state of incorporation', 'object': 'california', 'chunk': 1}, {'subject': 'apple inc.', 'predicate': 'irs employer id', 'object': '94-2404110', 'chunk': 1}, {'subject': 'apple inc.', 'predicate': 'address', 'object': 'one apple park way, cupertino, california 95014', 'chunk': 1}, {'subject': 'apple inc.', 'predicate': 'telephone number', 'object': '(408) 996-1010', 'chunk': 1}, {'subject': 'common stock, $0.00001 par value per share', 'predicate': 'trading symbol', 'object': 'aapl', 'chunk': 1}, {'subject': 'common stock, $0.00001 par value per share', 'predicate': 'exchange', 'object': 'the nasdaq stock market llc', 'chunk': 1}, {'subject': '0.000% notes due 2025', 'predicate': 'exchange', 'object': 'the nasdaq stock market llc', 'chunk': 1}]

In [410]:
# # Process all chunks
# print("\n=== STARTING EXTRACTION FOR ALL CHUNKS ===\n")

# for chunk_index in range(len(chunks)):
#     chunk_info = chunks[chunk_index]
#     chunk_text = chunk_info
#     chunk_num = chunk_index + 1
#     #
#     print(f"\n--- Processing Chunk {chunk_num}/{len(chunks)} --- ")
    
#     # 1. Format the User Prompt
#     print("1. Formatting User Prompt...")
#     user_prompt = extraction_user_prompt_template.format(text_chunk=chunk_text)
    
#     try:
#         # 2. Make the API Call
#         print("2. Sending request to LLM...")
        
#         # Send request using instructor client
#         response = client.messages.create(
#             messages=[
#                 {"role": "system", "content": extraction_system_prompt},
#                 {"role": "user", "content": user_prompt}
#             ],
#             response_model=TripleList
#         )
        
#         print("   LLM response received.")
        
#         # 3. Process Response
#         print("3. Processing response...")
        
#         # The response is already validated by instructor/pydantic
#         valid_triples_in_chunk = []
        
#         for triple in response.triples:
#             # Add source chunk info
#             triple_dict = triple.model_dump()
#             triple_dict['chunk'] = chunk_num
#             valid_triples_in_chunk.append(triple_dict)
        
#         # --- Display Valid Triples from this Chunk --- 
#         if valid_triples_in_chunk:
#             print(f"--- Valid Triples Extracted (Chunk {chunk_num}) ---")
#             display(pd.DataFrame(valid_triples_in_chunk))
#             print("-" * 20)
#             # Add to the main list
#             all_extracted_triples.extend(valid_triples_in_chunk)
#         else:
#             print(f"--- No valid triples extracted from this chunk. ---")
#             print("-" * 20)

#     except Exception as e:
#         error_message = str(e)
#         print(f"   ERROR during API call: {error_message}")
#         failed_chunks.append({
#             'chunk_number': chunk_num,
#             'error': f'API Error: {error_message}',
#             'response': ''
#         })

#     # --- Update Running Total (Visual Feedback) ---
#     print(f"--- Running Total Triples Extracted: {len(all_extracted_triples)} --- ")
#     print(f"--- Failed Chunks So Far: {len(failed_chunks)} --- ")
    
#     # Add a small delay between chunks to avoid rate limiting
#     time.sleep(1)

# # After processing all chunks, show summary and save results
# print("\n=== EXTRACTION COMPLETE ===")

# # --- Summary Statistics ---
# print(f"\n--- Overall Extraction Summary ---")
# print(f"Total chunks defined: {len(chunks)}")
# processed_chunks = len(chunks) - len(failed_chunks)
# print(f"Chunks processed successfully: {processed_chunks}")
# print(f"Total valid triples extracted: {len(all_extracted_triples)}")
# print(f"Number of failed chunks: {len(failed_chunks)}")

# # Print details of failed chunks if any
# if failed_chunks:
#     print("\nDetails of Failed Chunks:")
#     for failure in failed_chunks:
#         print(f"  Chunk {failure['chunk_number']}: {failure['error']}")

# # Create DataFrame of all triples
# all_triples_df = pd.DataFrame(all_extracted_triples)

# # Save results to CSV
# csv_filename = "extracted_triples.csv"
# all_triples_df.to_csv(csv_filename, index=False)
# print(f"\nSaved all triples to: {csv_filename}")

# # Save results to JSON
# results_dict = {
#     "metadata": {
#         "total_chunks": len(chunks),
#         "processed_chunks": processed_chunks,
#         "failed_chunks": len(failed_chunks),
#         "total_triples": len(all_extracted_triples),
#         "extraction_timestamp": datetime.datetime.now().isoformat()
#     },
#     "triples": all_extracted_triples,
#     "failed_chunks": failed_chunks
# }

# json_filename = "extraction_results.json"
# with open(json_filename, 'w', encoding='utf-8') as f:
#     json.dump(results_dict, f, indent=2, ensure_ascii=False)
# print(f"Saved complete results to: {json_filename}")

# # Display sample of extracted triples
# print("\n--- Sample of Extracted Triples (First 10) ---")
# if len(all_extracted_triples) > 0:
#     display(all_triples_df.head(10))
# else:
#     print("No triples were extracted.")

# print("\n=== PROCESS COMPLETE ===")

In [411]:
def normalize_and_deduplicate_triples(all_extracted_triples: List[dict]) -> List[dict]:
    """
    Normalize and deduplicate knowledge graph triples.
    
    Args:
        all_extracted_triples: List of extracted triples
        
    Returns:
        List of normalized and deduplicated triples
    """
    # Initialize lists and tracking variables
    normalized_triples = []
    seen_triples = set()  # Tracks (subject, predicate, object) tuples
    original_count = len(all_extracted_triples)
    empty_removed_count = 0
    duplicates_removed_count = 0

    print(f"Starting normalization and de-duplication of {original_count} triples...")
    print("Processing triples for normalization (showing first 5 examples):")
    example_limit = 5
    processed_count = 0

    for i, triple in enumerate(all_extracted_triples):
        show_example = (i < example_limit)
        if show_example:
            print(f"\n--- Example {i+1} ---")
            print(f"Original Triple (Chunk {triple.get('chunk', '?')}): {triple}")
        
        subject_raw = triple.get('subject')
        predicate_raw = triple.get('predicate')
        object_raw = triple.get('object')
        chunk_num = triple.get('chunk', 'unknown')
        
        triple_valid = False
        normalized_sub, normalized_pred, normalized_obj = None, None, None

        if isinstance(subject_raw, str) and isinstance(predicate_raw, str) and isinstance(object_raw, str):
            # 1. Normalize
            normalized_sub = subject_raw.strip().lower()
            normalized_pred = re.sub(r'\s+', ' ', predicate_raw.strip().lower()).strip()
            normalized_obj = object_raw.strip().lower()
            if show_example:
                print(f"Normalized: SUB='{normalized_sub}', PRED='{normalized_pred}', OBJ='{normalized_obj}'")

            # 2. Filter Empty
            if normalized_sub and normalized_pred and normalized_obj:
                triple_identifier = (normalized_sub, normalized_pred, normalized_obj)
                
                # 3. De-duplicate
                if triple_identifier not in seen_triples:
                    normalized_triples.append({
                        'subject': normalized_sub,
                        'predicate': normalized_pred,
                        'object': normalized_obj,
                        'source_chunk': chunk_num
                    })
                    seen_triples.add(triple_identifier)
                    triple_valid = True
                    if show_example:
                        print("Status: Kept (New Unique Triple)")
                else:
                    duplicates_removed_count += 1
                    if show_example:
                        print("Status: Discarded (Duplicate)")
            else:
                empty_removed_count += 1
                if show_example:
                    print("Status: Discarded (Empty component after normalization)")
        else:
            empty_removed_count += 1  # Count non-string/missing as needing removal
            if show_example:
                print("Status: Discarded (Non-string or missing component)")
        processed_count += 1

    print(f"\n... Finished processing {processed_count} triples.")
    print(f"\nNormalization Summary:")
    print(f"Original triples: {original_count}")
    print(f"Empty/invalid triples removed: {empty_removed_count}")
    print(f"Duplicate triples removed: {duplicates_removed_count}")
    print(f"Final unique triples: {len(normalized_triples)}")
    
    return normalized_triples


In [412]:
normalize_triples = normalize_and_deduplicate_triples(all_triples)

Starting normalization and de-duplication of 9 triples...
Processing triples for normalization (showing first 5 examples):

--- Example 1 ---
Original Triple (Chunk 1): {'subject': 'apple inc.', 'predicate': 'fiscal year ended', 'object': 'september 28, 2024', 'chunk': 1}
Normalized: SUB='apple inc.', PRED='fiscal year ended', OBJ='september 28, 2024'
Status: Kept (New Unique Triple)

--- Example 2 ---
Original Triple (Chunk 1): {'subject': 'apple inc.', 'predicate': 'commission file number', 'object': '001-36743', 'chunk': 1}
Normalized: SUB='apple inc.', PRED='commission file number', OBJ='001-36743'
Status: Kept (New Unique Triple)

--- Example 3 ---
Original Triple (Chunk 1): {'subject': 'apple inc.', 'predicate': 'state of incorporation', 'object': 'california', 'chunk': 1}
Normalized: SUB='apple inc.', PRED='state of incorporation', OBJ='california'
Status: Kept (New Unique Triple)

--- Example 4 ---
Original Triple (Chunk 1): {'subject': 'apple inc.', 'predicate': 'irs employer 

In [413]:
def define_schema(triples: List[dict]) -> Dict[str, str]:
    """
    Define schema by generating definitions for each unique relation using LLM.
    
    Args:
        triples: List of knowledge graph triples
        
    Returns:
        Dictionary mapping relations to their definitions
    """
    print("\nPhase 2: Defining Schema...")
    
    # Extract unique predicates from triples
    unique_relations = set(triple['predicate'] for triple in triples)
    relation_definitions = {}
    
    print(f"Found {len(unique_relations)} unique relations")
    print("Generating definitions using Gemini...")

    for relation in unique_relations:
        try:
            # Use the same Gemini client we already have configured
            prompt = f"""Given the relation '{relation}' extracted from a company's financial documents,
            provide a clear, concise one-sentence definition of what this relation means in the context
            of business and financial reporting. Focus on precision and clarity."""
            
            response = instructor_client.chat.completions.create(
                model="gemini-2.0-flash",
                messages=[{"role": "user", "content": prompt}],
                response_model=str,
                max_retries=3
            )
            
            relation_definitions[relation] = response.strip()
            print(f"✓ Defined: {relation}")
            
        except Exception as e:
            print(f"✗ Failed to define '{relation}': {str(e)}")
            relation_definitions[relation] = f"Undefined relation: {relation}"
    
    print(f"Schema definition complete. Defined {len(relation_definitions)} relations.")
    return relation_definitions

In [414]:
# define_schema(normalize_triples)

In [415]:
# from google import genai
# from google.genai import types
# import numpy as np
# client = genai.Client(api_key=API_KEY)

In [416]:
def get_embedding(text: str, task_type: str = "SEMANTIC_SIMILARITY") -> List[float]:
    """
    Get embedding for a text using Gemini's embedding model.
    
    Args:
        text: Text to embed
        task_type: Type of embedding task (SEMANTIC_SIMILARITY, CLUSTERING, etc.)
        
    Returns:
        List of floats representing the embedding
    """
    try:
        result = client.models.embed_content(
            model="gemini-embedding-exp-03-07",
            contents=text,
            config=types.EmbedContentConfig(task_type=task_type)
        )
        # The result.embeddings is a list of ContentEmbedding objects
        # We take the first embedding's values
        if result and result.embeddings and len(result.embeddings) > 0:
            return result.embeddings[0].values
        else:
            print(f"No embeddings found in response")
            return None
    except Exception as e:
        print(f"Error getting embedding for task {task_type}: {str(e)}")
        return None

def get_embeddings_batch(texts: List[str], task_type: str) -> List[List[float]]:
    """
    Get embeddings for a batch of texts.
    
    Args:
        texts: List of texts to embed
        task_type: Type of embedding task
        
    Returns:
        List of embedding vectors
    """
    embeddings = []
    for text in texts:
        embedding = get_embedding(text, task_type)
        if embedding is not None:
            embeddings.append(embedding)
        else:
            print(f"Warning: Could not get embedding for text: {text[:50]}...")
            embeddings.append([0] * 768)  # Use zero vector as fallback
    return embeddings

def compute_similarity(embedding1: List[float], embedding2: List[float]) -> float:
    """
    Compute cosine similarity between two embeddings.
    """
    a = np.array(embedding1)
    b = np.array(embedding2)
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def canonicalize_schema(triples: List[dict], relation_definitions: Dict[str, str], 
                        cluster_threshold: float = 0.75, 
                        semantic_threshold: float = 0.8) -> Tuple[List[dict], Dict[str, str]]:
    """
    Canonicalize schema by finding similar relations using both clustering and similarity embeddings.
    
    Args:
        triples: List of knowledge graph triples
        relation_definitions: Dictionary of relation definitions
        cluster_threshold: Threshold for clustering similarity (default: 0.75)
        semantic_threshold: Threshold for semantic similarity (default: 0.8)
        
    Returns:
        Tuple of (canonicalized triples, mapping of original to canonical relations)
    """
    print(f"\nPhase 3: Canonicalizing Schema (Cluster Threshold: {cluster_threshold}, Semantic Threshold: {semantic_threshold})...")
    
    try:
        definitions = list(relation_definitions.values())
        relation_keys = list(relation_definitions.keys())
        
        # Get embeddings optimized for clustering (to find groups)
        print("Generating clustering embeddings...")
        clustering_embeddings = get_embeddings_batch(definitions, "CLUSTERING")
        
        # Get embeddings optimized for similarity (to find exact matches)
        print("Generating similarity embeddings...")
        similarity_embeddings = get_embeddings_batch(definitions, "SEMANTIC_SIMILARITY")
        
        # Initialize canonical mapping
        canonical_map = {}
        processed = set()
        
        # Track similarity scores for analysis
        all_scores = []
        
        # Process each relation
        print("\nFinding similar relations...")
        for i, relation in enumerate(relation_keys):
            if relation in processed:
                continue
                
            similar_relations = []
            
            # Find similar relations using both embedding types
            for j, other_relation in enumerate(relation_keys):
                if i != j and other_relation not in processed:
                    # Check both clustering and similarity scores
                    cluster_sim = compute_similarity(clustering_embeddings[i], clustering_embeddings[j])
                    semantic_sim = compute_similarity(similarity_embeddings[i], similarity_embeddings[j])
                    
                    # Track scores for analysis
                    all_scores.append({
                        'relation1': relation,
                        'relation2': other_relation,
                        'cluster_sim': cluster_sim,
                        'semantic_sim': semantic_sim,
                        'avg_sim': (cluster_sim + semantic_sim) / 2
                    })
                    
                    # Use clustering score for grouping and semantic score for confirmation
                    if cluster_sim > cluster_threshold and semantic_sim > semantic_threshold:
                        similar_relations.append((other_relation, (cluster_sim + semantic_sim) / 2))
            
            # If we found similar relations, create a group
            if similar_relations:
                # Sort by average similarity score
                similar_relations.sort(key=lambda x: x[1], reverse=True)
                
                # Use the current relation as canonical form
                canonical_relation = relation
                processed.add(canonical_relation)
                canonical_map[canonical_relation] = canonical_relation
                
                # Map similar relations to the canonical form
                for similar_rel, score in similar_relations:
                    canonical_map[similar_rel] = canonical_relation
                    processed.add(similar_rel)
                    print(f"Mapped '{similar_rel}' → '{canonical_relation}' (similarity: {score:.2f})")
            else:
                # No similar relations found, use as its own canonical form
                canonical_map[relation] = relation
                processed.add(relation)
        
        # Update triples with canonical relations
        print("\nUpdating triples with canonical relations...")
        canonical_triples = []
        for triple in triples:
            canonical_triple = triple.copy()
            canonical_triple['predicate'] = canonical_map[triple['predicate']]
            canonical_triples.append(canonical_triple)
        
        # Summary statistics
        original_relations = len(set(t['predicate'] for t in triples))
        canonical_relations = len(set(t['predicate'] for t in canonical_triples))
        
        # Analyze similarity scores
        if all_scores:
            avg_cluster_sim = sum(s['cluster_sim'] for s in all_scores) / len(all_scores)
            avg_semantic_sim = sum(s['semantic_sim'] for s in all_scores) / len(all_scores)
            
            print(f"\nSimilarity Analysis:")
            print(f"- Average clustering similarity: {avg_cluster_sim:.3f}")
            print(f"- Average semantic similarity: {avg_semantic_sim:.3f}")
            print(f"- Pairs above thresholds: {len([s for s in all_scores if s['cluster_sim'] > cluster_threshold and s['semantic_sim'] > semantic_threshold])}")
        
        print(f"\nCanonicalization complete:")
        print(f"- Original unique relations: {original_relations}")
        print(f"- Canonical unique relations: {canonical_relations}")
        print(f"- Relations merged: {original_relations - canonical_relations}")
        
        return canonical_triples, canonical_map
        
    except Exception as e:
        print(f"Error during canonicalization: {str(e)}")
        return triples, {r: r for r in relation_definitions.keys()}

In [417]:
# Phase 2: Define schema
relation_definitions = define_schema(normalize_triples)

# Phase 3: Canonicalize schema
canonical_triples, canonical_map = canonicalize_schema(normalize_triples, relation_definitions)



Phase 2: Defining Schema...
Found 8 unique relations
Generating definitions using Gemini...


✓ Defined: telephone number
✓ Defined: trading symbol
✓ Defined: irs employer id
✓ Defined: commission file number
✓ Defined: address
✓ Defined: exchange
✓ Defined: fiscal year ended
✓ Defined: state of incorporation
Schema definition complete. Defined 8 relations.

Phase 3: Canonicalizing Schema (Cluster Threshold: 0.75, Semantic Threshold: 0.8)...
Generating clustering embeddings...
Generating similarity embeddings...

Finding similar relations...
Mapped 'address' → 'telephone number' (similarity: 0.90)
Mapped 'commission file number' → 'telephone number' (similarity: 0.87)
Mapped 'fiscal year ended' → 'telephone number' (similarity: 0.85)
Mapped 'exchange' → 'telephone number' (similarity: 0.84)
Mapped 'irs employer id' → 'telephone number' (similarity: 0.84)
Mapped 'trading symbol' → 'telephone number' (similarity: 0.83)

Updating triples with canonical relations...

Similarity Analysis:
- Average clustering similarity: 0.872
- Average semantic similarity: 0.827
- Pairs above thres

In [418]:


result = client.models.embed_content(
        model="gemini-embedding-exp-03-07",
        contents="What is the meaning of life?",
        config=types.EmbedContentConfig(task_type="SEMANTIC_SIMILARITY")
)
print(result.embeddings)

[ContentEmbedding(values=[-0.029910328, -0.01064506, 0.01232048, -0.04363388, -0.013421752, 0.002329359, -0.0057898476, 0.01377629, 0.042485517, -0.0057607763, 0.016976383, -0.016454414, -0.024323622, 0.04008316, 0.13324209, 0.0067314673, -0.006783664, 0.011491284, 0.014296529, -0.006029128, 0.008233743, 0.00345852, 0.002163263, -0.012119229, -0.014921178, -0.0056867404, 0.01915917, 0.00018566486, 0.014377131, -0.006765622, -0.0035427376, 0.019510606, 0.005653181, 0.030145729, -0.009636214, -0.0028940549, 0.017069276, 0.006264033, -7.985408e-05, 0.021383405, -0.024914917, -0.00023122504, 0.0043579782, 0.0029407823, 0.032731887, -0.00259939, 0.0068594706, -0.027191132, -0.0029506884, 0.020990087, -0.015325378, 0.024030503, -0.0023436325, -0.16248043, -0.0033473268, 0.011838764, -0.006498432, -0.007640825, -0.0030167769, -0.0036708876, -0.021138562, -0.0042642243, -0.018381873, -0.04532587, 0.00734486, -0.009865271, 0.0018252932, -0.0019786523, -0.012262748, -0.0014146615, -0.015477954, 

In [419]:
 #Extract text from PDF
pdf_path = "10K-filings/10K-filings/AAPL_10k.pdf"

# Extract company name from PDF filename
company_name = os.path.basename(pdf_path).split('_')[0]  # Gets 'AAPL' from 'AAPL_10k.pdf'
print(f"Processing 10-K filing for company: {company_name}")


Processing 10-K filing for company: AAPL
